In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/MachineLearning/data/mnist/train.csv')
display(df.head())

In [ ]:
from sklearn.metrics import classification_report

y_true = [0, 1, 1, 2, 2]
y_pred = [0, 0, 2, 2, 1]

target_name = ['thin', 'normal', 'fat']

print(classification_report(y_true, y_pred, target_names=target_name))

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]

confusion_matrix(y_true, y_pred)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix

#### 텐서플로우1로 구현하기

## 1. 데이터 불러와~
df = pd.read_csv('/content/drive/My Drive/MachineLearning/mnist/train.csv')
# display(df.head())

## 2. 결측치 이상치 확인
# 얘는 없어

## 3. 데이터 확인해보기 - 어떤 이미지일까?
img_data = df.drop('label', axis=1, inplace=False).values  # 이미지의 픽셀 데이터만 ndarray형식으로
fig = plt.figure()  # 도화지 깔어~
fig_arr = list()  # 나눌 영억을 저장할 리스트

for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28), cmap='Greys', 
                      interpolation='nearest')

plt.tight_layout()
plt.show()

## 4. data split
# 데이터는 크게 3부분으로 train(완성), validation(수정), test(검사)
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1), df['label'], test_size=0.3, 
                 random_state=0)

## 5. nomalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
del x_data_train
del x_data_test

## 6. t_data(label)를 one hot형태로
sess = tf.Session()

t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))

##################################################################
# 준비완료
##################################################################

### Tensorflow 구현

## 1. placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

## 2. Weight, bias
W = tf.Variable(tf.random.normal([784,10]), name='weight')
b = tf.Variable(tf.random.normal([10]), name='bias')

## 3. Model(Hypothesis) => Multinomial
logit = tf.matmul(X,W) + b  # linear regression
H = tf.nn.softmax(logit)  # Multinomial Hypothesis

## 4. loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

## 5. Optimizer를 이용한 train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

## 6. 학습용 parameter setting (기본적으로 2개는 설정)
num_of_epoch = 100
batch_size = 100

## 7. 학습진행
def run_train(sess, train_x, train_t):
    print('#### 학습시작 ####')
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(train_x.shape[0] / batch_size)
    for step in range(num_of_epoch):
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train, loss], feed_dict={X:batch_x, 
                                                             T:batch_t})
        if step % 10 == 0:
            print('loss : {}'.format(loss_val))
    print('#### 학습종료 ####')

## Accuracy
predict = tf.argmax(H,1)
# sklearn을 이용해서 classification_report를 출력해보자
target_name = ['num 0', 'num 1', 'num 2', 'num 3', 
               'num 4', 'num 5', 'num 6', 'num 7', 
               'num 8', 'num 9']
run_train(sess, x_data_train_norm, t_data_train_onehot)




In [ ]:
## 성능평가

print(classification_report(t_data_train, 
                            sess.run(predict, feed_dict={X:x_data_train_norm}), 
                            target_names=target_name))

In [ ]:
## seaborn을 이용한 confusion matrix의 그래프 출력
fig, ax = plt.subplots(figsize=(10,10))

sns.heatmap(
    confusion_matrix(t_data_train, 
                     sess.run(predict, feed_dict={X:x_data_train_norm})), 
    annot = True,  # 숫자표현
    cbar = True,  # 컬러바
    fmt = '3d',  # 정수표현
    cmap = 'Blues',  # 컬러색상
    ax = ax  #그래프로  사용할 subplot
)

ax.set_xlabel('Predict')
ax.set_ylabel('Actual')
plt.show()